In [1]:
import os
import sys
import psycopg2
from sqlalchemy import create_engine
from io import StringIO
import pandas as pd
import csv
up_one =  (os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
sys.path.append(up_one)
up_two =  (os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)))
sys.path.append(up_two)
#from minorlab import metapdb
import subprocess
from collections import defaultdict

import pprint
pp = pprint.PrettyPrinter(indent=4)




## Export cell output to a file 

Cell 1  
<code>
    %%capture cap --no-stderr
    print(q)
</code>

Cell 2  
<code>
with open('output.txt', 'w') as f:    
    f.write(cap.stdout)
    </code>

### Appris files and function

In [2]:
appris = pd.read_csv('Data/appris_transcript_names.tsv', sep='\t', header=None, names=['gene','ref'])
nonappris = pd.read_csv('Data/not_appris_transcript_names.tsv', sep='\t', header=None, names=['gene','notref'])
def ref_nonref(gene):
    ref = appris.loc [ appris['gene']== gene]['ref'].tolist()
    #print (ref)
    non_ref = nonappris.loc [nonappris['gene']==gene]['notref'].tolist()
    non_ref.sort()
    #print (non_ref)
    return (ref, non_ref)

In [3]:
print (ref_nonref('TPM3'))

(['TPM3-224'], ['TPM3-201', 'TPM3-202', 'TPM3-203', 'TPM3-204', 'TPM3-205', 'TPM3-206', 'TPM3-207', 'TPM3-208', 'TPM3-209', 'TPM3-210', 'TPM3-211', 'TPM3-212', 'TPM3-213', 'TPM3-214', 'TPM3-215', 'TPM3-216', 'TPM3-217', 'TPM3-218', 'TPM3-219', 'TPM3-220', 'TPM3-221', 'TPM3-222', 'TPM3-223', 'TPM3-225', 'TPM3-226', 'TPM3-227'])


In [4]:
## Read in full blast results
# Note version 1 of blast_full_PDB_vs_gencode had some lines that were Warnings
columns =  ['qseqid','qlen', 'qstart', 'qend', 'sseqid', 'slen', 'sstart', 'send', 'evalue', 'bitscore', 'length', 'nident', 'gaps','gapopen', 'mismatch', 'pident']
blast_df= pd.read_csv('Data/blast_full_PDB_vs_gencode_v2.tsv', sep='\t', header = 1, names = columns)
blast_df

,qseqid,qlen,qstart,qend,sseqid,slen,sstart,send,evalue,bitscore,length,nident,gaps,gapopen,mismatch,pident
1,7evw_R,599,36,574,FZD1-201|ENST00000287934.4,647,103,647,0.000000e+00,905.0,551,440,18,6,93,79.855
2,7evw_R,599,38,574,FZD2-201|ENST00000315323.5,565,28,565,0.000000e+00,893.0,553,434,31,3,88,78.481
3,7evw_R,599,49,563,FZD5-201|ENST00000295417.4,585,33,536,2.420000e-180,523.0,533,276,47,14,210,51.782
4,7evw_R,599,44,589,FZD10-201|ENST00000229030.5,581,29,573,1.480000e-165,485.0,562,261,33,10,268,46.441
5,7evw_R,599,49,562,FZD3-201|ENST00000240093.8,666,28,512,2.230000e-162,480.0,515,236,31,3,248,45.825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1909511,"2z9a_A,B",88,3,81,PSAP-201|ENST00000394936.8,524,311,389,7.630000e-47,157.0,79,79,0,0,0,100.000
1909512,"2z9a_A,B",88,3,55,PSAP-204|ENST00000633965.1,167,115,167,4.070000e-31,108.0,53,53,0,0,0,100.000
1909513,8d58_A,246,1,246,METTL1-202|ENST00000324871.12,276,20,265,0.000000e+00,516.0,246,246,0,0,0,100.000
1909514,8d58_A,246,104,240,METTL1-204|ENST00000548504.1,124,2,124,3.650000e-85,250.0,137,123,14,1,0,89.781


### Function to reduce the data to top matches of each PDB
Function is called top_five, the the second option is the numner of hits to include

In [ ]:
def top_five(df, max=5 ):
    keep= []
    last = ""
    for i,row in df.iterrows():
        if row['qseqid'] != last:
            count=1
            last = row['qseqid']
            keep.append(True)
        else:
            count += 1
            if count<=max:
                keep.append(True)
            else:
                keep.append(False)
    return (df.loc[keep])
#top_five_df = top_five(copy_df)
#top_five_df.to_excel('top5_pdb_vs_gencode.xlsx')

In [ ]:
def top_five_bitscore(df, max=5 ):
    keep= []
    last = ""
    for i,row in df.iterrows():
        if row['qseqid'] != last:
            count=1
            last = row['qseqid']
            bitscore_threshold  = row['bitscore'] * 0.95
            keep.append(True)
        else:
            count += 1
            if count<=max and row['bitscore'] > bitscore_threshold:
                keep.append(True)
            else:
                keep.append(False)
    return (df.loc[keep])
bitscore_df = top_five_bitscore(blast_df)
bitscore_df
#top_five_df.to_excel('top5_pdb_vs_gencode.xlsx')

## This will find all isoforms that 
def top_five_bitscore_exact(df, max=5 ):
    keep= []
    last = ""
    for i,row in df.iterrows():
        if row['qseqid'] != last:
            count=1
            last = row['qseqid']
            bitscore_threshold  = row['bitscore'] 
            keep.append(True)
        else:
            count += 1
            if row['bitscore'] == bitscore_threshold and count<=max:
                keep.append(True)
            else:
                keep.append(False)
    return (df.loc[keep])

In [ ]:
top_100_matched = top_five_bitscore_exact(blast_df, 100)
top_100_matched

In [ ]:
top_100_list = []
for i,row in top_100_matched.iterrows():
    top_100_list.append(row.tolist())
print (top_100_list[0])

In [ ]:
print (top_100_list[0])

In [ ]:
top_100_matched.to_csv('Data/top_100_matched.tsv', sep='\t')

In [ ]:
pdb_to_gencode[0]
#pdb_grouped_by_qseqid

In [ ]:
pdb_to_gencode = []
with open('Data/blast_full_PDB_vs_gencode.tsv', 'r') as pdb_file:
    pdb_reader = csv.reader(pdb_file, delimiter='\t')
    next(pdb_reader, None)
    for row in pdb_reader:
        row[5] = row[5].split('|')[0]
        pdb_to_gencode.append(row)
    print (len (pdb_to_gencode))
print (pdb_to_gencode[0:2])

In [11]:
# Read in the appris transcripts
appris_transcripts = set()
with open('Data/appris_transcript_names.tsv', 'r') as appris_file:
    appris_reader = csv.reader(appris_file, delimiter='\t')
    for row in appris_reader:
        appris_transcripts.add(row[1])
    print (f'{len(appris_transcripts)} Appris transcripts')

# Read in the nonappris transcripts
nonappris_transcripts = set()
with open('Data/not_appris_transcript_names.tsv', 'r') as nonappris_file:
    nonappris_reader = csv.reader(nonappris_file, delimiter='\t')
    for row in nonappris_reader:
        nonappris_transcripts.add(row[1])
    print (f'{len(nonappris_transcripts)} non-Appris transcripts')

# Read in the PDB to GenCode mapping
pdb_to_gencode = []
with open('Data/blast_full_PDB_vs_gencode_v2.tsv', 'r') as pdb_file:
    pdb_reader = csv.reader(pdb_file, delimiter='\t')
    next(pdb_reader, None)
    for row in pdb_reader:
        # This makes the sseqid just the isoform name
        row[5] = row[5].split('|')[0]
        pdb_to_gencode.append(row)
    print (f'{len (pdb_to_gencode)} records in Gencode')

    


80169 Appris transcripts
141431 non-Appris transcripts
1909443 records in Gencode


In [15]:
# Group the PDB to GenCode mappings by qseqid
# print (pdb_to_gencode[0])
pdb_grouped_by_qseqid = {}
for row in pdb_to_gencode:
    #print (row)
    qseqid = row[1]
    if qseqid not in pdb_grouped_by_qseqid:
        pdb_grouped_by_qseqid[qseqid] = []
    pdb_grouped_by_qseqid[qseqid].append(row)
print(f' This represents {len(pdb_grouped_by_qseqid)} pdbs')
# Below is just a check.
for k in list(pdb_grouped_by_qseqid.keys())[0:2]:
   print (pdb_grouped_by_qseqid[k])
print (len (pdb_grouped_by_qseqid))

 This represents 35081 pdbs
[['0', '7evw_R', '599', '38', '574', 'FZD7-201', '574', '38', '574', '0.0', '1115', '537', '537', '0', '0', '0', '100.000'], ['1', '7evw_R', '599', '36', '574', 'FZD1-201', '647', '103', '647', '0.0', '905', '551', '440', '18', '6', '93', '79.855'], ['2', '7evw_R', '599', '38', '574', 'FZD2-201', '565', '28', '565', '0.0', '893', '553', '434', '31', '3', '88', '78.481'], ['3', '7evw_R', '599', '49', '563', 'FZD5-201', '585', '33', '536', '2.42e-180', '523', '533', '276', '47', '14', '210', '51.782'], ['4', '7evw_R', '599', '44', '589', 'FZD10-201', '581', '29', '573', '1.48e-165', '485', '562', '261', '33', '10', '268', '46.441'], ['5', '7evw_R', '599', '49', '562', 'FZD3-201', '666', '28', '512', '2.23e-162', '480', '515', '236', '31', '3', '248', '45.825'], ['6', '7evw_R', '599', '49', '562', 'FZD3-204', '666', '28', '512', '2.23e-162', '480', '515', '236', '31', '3', '248', '45.825'], ['7', '7evw_R', '599', '49', '563', 'FZD6-201', '706', '24', '509', '2.

In [16]:
# Filter the PDB to GenCode mapping based on appris and nonappris transcripts
filtered_pdb_to_gencode = []
for qseqid, pdb_group in pdb_grouped_by_qseqid.items():
    #print (f'qseqid, pdb_group = {qseqid}, {pdb_group}')
    
    # Get the top hit based on bitscore
    top_hit = max(pdb_group, key=lambda row: float(row[10]))
    #print (f'top_hit = {top_hit}')
    top_bitscore = float(top_hit[10])
    
    # Get all other hits with the same bitscore as the top hit
    other_hits = [row for row in pdb_group if float(row[10]) == top_bitscore and row != top_hit]
    
    # Determine if all sseqids are nonappris-
    all_nonappris = all(row[5] in nonappris_transcripts for row in pdb_group)
    
    # If all sseqids are nonappris, add the gene name to each row and output to the filtered list
    if all_nonappris:
        top_hit_with_gene = top_hit + [top_hit[5].split('-')[0]]
        filtered_pdb_to_gencode.append(top_hit_with_gene)
        for other_hit in other_hits:
            other_hit_with_gene = other_hit + [other_hit[5].split('-')[0]]
            filtered_pdb_to_gencode.append(other_hit_with_gene)
print (f'filtered_pdb_to_gencode is {len(filtered_pdb_to_gencode)} long')

print (filtered_pdb_to_gencode[0:3])

filtered_pdb_to_gencode is 53 long
[['61548', '6otn_A,B,C,D', '81', '3', '81', 'TPM3-205', '248', '2', '80', '8.42e-44', '142', '79', '78', '0', '0', '1', '98.734', 'TPM3'], ['61549', '6otn_A,B,C,D', '81', '3', '81', 'TPM3-204', '247', '2', '80', '8.75e-44', '142', '79', '78', '0', '0', '1', '98.734', 'TPM3'], ['61550', '6otn_A,B,C,D', '81', '3', '81', 'TPM3-208', '247', '2', '80', '8.75e-44', '142', '79', '78', '0', '0', '1', '98.734', 'TPM3']]


In [58]:
combined_filtered =  [sublist for sublist in filtered_pdb_to_gencode ]
#print (combined_filtered)
columns =  ['index','qseqid','qlen', 'qstart', 'qend', 'sseqid', 'slen', 'sstart', 'send', 'evalue', 'bitscore', 'length', 'nident', 'gaps','gapopen', 'mismatch', 'pident', 'gene']
non_appris_pdbs =pd.DataFrame(combined_filtered, columns=columns)
non_appris_pdbs =non_appris_pdbs.drop(columns=['index'])
non_appris_pdbs = non_appris_pdbs.sort_values(by=['gene','sseqid','qseqid'])
non_appris_pdbs

,qseqid,qlen,qstart,qend,sseqid,slen,sstart,send,evalue,bitscore,length,nident,gaps,gapopen,mismatch,pident,gene
11,"4or5_E,J",43,6,43,AFF4-204,148,32,69,8.81e-22,82.0,38,38,0,0,0,100.000,AFF4
44,2lmb_A,43,1,43,AGER-205,294,252,294,7.92e-21,82.4,43,42,0,0,1,97.674,AGER
45,"4zp3_M,N,O,P,Q,R",40,1,40,AKAP7-205,104,43,82,1.29e-22,82.8,40,40,0,0,0,100.000,AKAP7
50,2ksg_A,48,1,48,DCD-201,110,63,110,5.72e-25,89.4,48,48,0,0,0,100.000,DCD
16,"5nvm_B,D",42,5,42,GIGYF2-209,99,35,72,3.93e-21,79.0,38,38,0,0,0,100.000,GIGYF2
19,"5nvm_B,D",42,5,42,GIGYF2-212,115,35,72,5.48e-21,79.0,38,38,0,0,0,100.000,GIGYF2
17,"5nvm_B,D",42,5,42,GIGYF2-213,106,35,72,4.01e-21,79.0,38,38,0,0,0,100.000,GIGYF2
18,"5nvm_B,D",42,5,42,GIGYF2-214,112,35,72,5.07e-21,79.0,38,38,0,0,0,100.000,GIGYF2
24,6k06_A,53,2,53,GOLGA2-219,125,9,60,1.07e-21,81.6,52,48,0,0,4,92.308,GOLGA2
23,6k06_A,53,2,53,GOLGA2-225,89,9,60,4.39e-22,81.6,52,48,0,0,4,92.308,GOLGA2


In [54]:
npp_g2=non_appris_pdbs.groupby('gene', as_index = False) ['qseqid'].nunique()
npp_g2 = npp_g2.rename(columns={'qseqid':'count'}).sort_values(by=['count'], ascending =False)
npp_g2

,gene,count
16,PRKAR2A,3
29,YAP1,2
28,VASP,2
8,INS,2
27,UBL7,1
26,TPM3,1
25,TP73,1
24,TNC,1
23,TCF3,1
22,SNUPN,1


In [53]:
npp_g=non_appris_pdbs.groupby('gene', as_index = False) ['sseqid'].nunique()
npp_g = npp_g.rename(columns={'sseqid':'count'}).sort_values(by=['count'], ascending =False)
npp_g

,gene,count
26,TPM3,6
4,GIGYF2,4
9,ITGB1,3
16,PRKAR2A,3
14,PAK6,3
5,GOLGA2,3
25,TP73,2
29,YAP1,2
6,GYPA,2
18,RAB11FIP3,1


### Function to extract sequences of Isoforms and related sequences
USE: get_gene_seqs (gene, df=found_df)  
By default, will extract data from found_df, but can be specified  
##### default_df is currently non_appris_pdbs 


In [33]:
pdb_file='Data/pdb_unique_human_seqs_230301.fa'
gene_file='Data/gencode_v43_translations_mod.fa'
default_df = non_appris_pdbs
def extract_sequence(filename, sequence_id):
    sequence = ""
    with open(filename, "r") as f:
        for line in f:
            if line.startswith(">") and sequence_id in line:
                sequence += line
                # Found the sequence ID we want
                while True:
                    line = f.readline().strip()
                    if line.startswith(">") or not line:
                        # Reached the end of the sequence
                        break
                    sequence += line
    sequence += "\n"
    return sequence
def get_gene_seqs (gene, df=default_df):
    matches = df.loc[ df['sseqid'].str.contains(gene) ]
    isos = set()
    pdbs = set()
    for i,row in matches.iterrows():
        isos.add(row['sseqid'])
        pdbs.add(row['qseqid'])
    seqs=""
    for pdb in pdbs:
        seqs+= extract_sequence (pdb_file, pdb)
    for iso in isos:
        seqs+= extract_sequence (gene_file, iso)
    return (seqs)

s=get_gene_seqs('GIGYF2')
print (s)

>5nvm_B,D
GPHMKYKLADYRYGREEMLALFLKDNKIPSDLLDKEFLPILQ
>GIGYF2-209|ENST00000425040.5
MAAETQTLNFGPEWLRALSSGGSITSPPLSPALPKYKLADYRYGREEMLALFLKDNKIPSDLLDKEFLPILQEEPLPPLALVPFTEEEQRNFSMSVNSA
>GIGYF2-213|ENST00000429187.5
MAAETQTLNFGPEWLRALSSGGSITSPPLSPALPKYKLADYRYGREEMLALFLKDNKIPSDLLDKEFLPILQEEPLPPLALVPFTEEEQRNFSMSVNSAAVLRLTG
>GIGYF2-214|ENST00000430720.5
MAAETQTLNFGPEWLRALSSGGSITSPPLSPALPKYKLADYRYGREEMLALFLKDNKIPSDLLDKEFLPILQEEPLPPLALVPFTEEEQRNFSMSVNSAAVLRLTGRGGGGT
>GIGYF2-212|ENST00000428883.5
MAAETQTLNFGPEWLRALSSGGSITSPPLSPALPKYKLADYRYGREEMLALFLKDNKIPSDLLDKEFLPILQEEPLPPLALVPFTEEEQRNFSMSVNSAAVLRLTGRGGGGTVVG



### Create a directory with sequence files for all the matches

In [36]:
for gene in set (non_appris_pdbs['gene'].tolist()):
    file = f'Data/Non_appris_seqs/{gene}.fa'
    if os.path.isfile(file):
        continue
    s = get_gene_seqs(gene)
    with open (file, 'w') as out:
        out.write(s)
    

In [38]:
data_dir = 'Data/Non_appris_seqs'
def show_msa(gene_list, fresh=0, last=None):
    '''Will generate and show a MSA for a list of genes. If last is set, it will skip until it reaches that gene'''
    if len(gene_list)==1:
        fresh =1
    for g in gene_list:
        if fresh==0 and g!=last:
            continue
        print (g)
        print (f'Reference isoform is {ref_nonref(g)[0]}')
        fresh=1
        last = g
        fasta_file = f'{data_dir}/{g}.fa'
        if not os.path.isfile(fasta_file):
            s=get_gene_seqs(g)
            with open (fasta_file, 'w') as out:
                out.write(s)
        msa_file = f'{data_dir}/{g}.msa'
        if not os.path.isfile(msa_file):
            cmd = f'clustalo -i {fasta_file} --outfmt=clu -o {msa_file}'
            print (cmd)
            r=subprocess.getoutput(cmd)
            if r[0:5] =="ERROR":
                print (f'Something went wrong with {g}')
                continue
            print (r)
        with open (msa_file, 'r') as f:
            rr=f.read()
            print (rr)

        #read = input ("Ready to move on? f to see sequence file ")
        #if read.lower() == 'f':
            with open (fastafile) as seqs:
                print (seqs.read())
            if len (gene_list) > 1:
                read = input ("Ready to move on? ")
            
def show_seqs_loop(gene_list, fresh=0, last=None):
    #expects a df with the name of the gene in position 0
    for g in gene_list:
        #print (g)
        if fresh==0 and g!=last:
            continue
        fresh=1
        last = g   
        with open (f'Data/Seq_groups/{g}.fa') as seqs:
            print(seqs.read())
        read = input ("Ready to move on? f to see sequence file")

def show_seqs(gene):
    with open (f'Data/Seq_groups/{gene}.fa') as seqs:
        print (seqs.read())

def matches_for_gene(gene, df = default_df):
    df2 = df.loc[df['sseqid'].str.startswith(gene)]
    return (df2)

In [ ]:
show_msa(to_inspect, fresh =0, last='EPHA7')

In [ ]:
show_msa(['HARS1'])

In [ ]:
functional=pd.read_csv('Data/APPRIS_FunctionalTableLong.tsv',sep ='\t')
functional_genes = functional['HGNC'].tolist()
functional_genes
show_msa(functional_genes,fresh=0, last='ITGA7')


In [ ]:
functional=pd.read_csv('Data/Tandem_exon_derived_substitution_HES_APPRIS_functional.tsv',sep ='\t')
functional_genes = functional['HGNC'].tolist()
functional_genes
show_msa(functional_genes,fresh=1)


# Below here extra   
mirage, exonerate


In [ ]:
blast_df.groupby(['qseqid'])['sseqid'].aggregate(lambda x: list(x)).reset_index()




## Identify genes with more than one isoform. 

In [ ]:
genes_dict={}
def multiple_isos(df):
    genes=defaultdict(set)
    # Build a dictionary where the values are a set of isoforms for each gene
    for id in df['sseqid']:
        try:
            id = id.split('|')[0]
            parts = id.split('-')
            gene='-'.join(parts[0:-1])
            iso=parts[-1]
            #print (id, gene, iso)
            genes[gene].add(iso)
        except:
            pass
    # Create a list (check) of genes that have more than 1 isoform represented.
    check = []
    for g,g_set in genes.items():
        if len(g_set) > 1:
            check.append(g)
            genes_dict[g]=g_set
    print (f'check is {len(check)} long')

    # Create list of df indicies of all genes that have more than one df 
    found=[]
    for i,row in df.iterrows():
        #print (row['sseqid'].split('-')[0])
        try:
            if row['sseqid'].split('-')[0] in check:
                found.append(i)
        except:
            pass
    #print (f'found is {found}')
    found_df = df.loc[found]
    found_df.sort_values(by=['sseqid'], inplace=True)
    return found_df

found_df = multiple_isos(top_100_matched)
found_df

### Find genes that have a structures that match ref and non-ref isoforms 
Note: genes_dict will only contain genes that match more than one isoform.

In [ ]:
to_inspect=[]
#for k in list(genes_dict.keys())[0:10]:
for k in genes_dict:
    try:
        r,nr=ref_nonref(k)
        r_iso = r.split('-')[1]
        if r_iso in genes_dict[k]:
            to_inspect.append(k)
    except Exception as e:
        pass

    

In [ ]:
columns =  ['qseqid','qlen', 'qstart', 'qend', 'sseqid', 'slen', 'sstart', 'send', 'evalue', 'bitscore', 'length', 'nident', 'gaps','gapopen', 'mismatch', 'pident']
filtered = pd.read_csv('Data/filtered_pdb_gencode_mapping.tsv', sep='\t', index_col= False, header = None, names = columns)
filtered

### Write out 

In [ ]:
# Write out the filtered mapping to a TSV file
with open('Data/filtered_pdb_gencode_mapping.tsv', 'w') as output_file:
    output_writer = csv.writer(output_file, delimiter='\t')
    for row in filtered_pdb_to_gencode:
        output_writer.writerow(row)

In [ ]:
gene_dict= defaultdict(set)
def seq_loop(df, dir ='Data/Seq_groups/' ):
    gene_list = []
    for id in df['sseqid']:
        try:
            id = id.split('|')[0]
            parts = id.split('-')
            gene='-'.join(parts[0:-1])
            if gene not in gene_list:
                gene_list.append(gene)
                gene_dict[gene].add(id)
                file = f'Data/Seq_groups/{gene}.fa'
                if os.path.isfile(file):
                    continue
                s = get_gene_seqs(gene)
                with open (f'{dir}/{gene}.fa', 'w') as out:
                    out.write(s)
        except:
            pass
    return (gene_list)
gene_list=seq_loop(found_df, dir)
#filtered_genes= pd.DataFrame(done)  
    